In [1]:

%pprint

Pretty printing has been turned OFF


In [2]:

%%time
%run ../py/html_analysis.py
ha = HeaderAnalysis()

Wall time: 4.05 s


In [3]:

import sqlite3
import os

db = sqlite3.connect(os.path.join('../', 'instance', 'flaskr.sqlite'), detect_types=sqlite3.PARSE_DECLTYPES)
db.row_factory = sqlite3.Row

In [4]:

%run ../py/sql_utlis.py
su = SqlUtilities()

Wall time: 29 ms


In [ ]:

def update_remaining_child_strs_list_dictionary(child_strs_list_dict=None, navigable_parent_is_header_dict=None):
    if navigable_parent_is_header_dict is None:
        navigable_parent_is_header_dict = ha.NAVIGABLE_PARENT_IS_HEADER_DICT
    if child_strs_list_dict is None:
        files_list = os.listdir(ha.SAVES_HTML_FOLDER)
        child_strs_list_dict = {}
        for file_name in files_list:
            child_strs_list = ha.get_child_strs_from_file(file_name)
            if not len(child_strs_list):
                os.remove(file_path)
            child_strs_list = [child_str for child_str in child_strs_list if child_str not in navigable_parent_is_header_dict]
            child_strs_list_dict[file_name] = child_strs_list
        
        return child_strs_list_dict
    
    for file_name, child_strs_list in child_strs_list_dict.items():
        child_strs_list = [child_str for child_str in child_strs_list if child_str not in navigable_parent_is_header_dict]
        if not len(child_strs_list):
            child_strs_list = child_strs_list_dict.pop(file_name)
            break
        else:
            child_strs_list_dict[file_name] = child_strs_list
    
    return child_strs_list_dict


----
'TASK_SCOPE_HEADERS_LIST', 'REQ_QUALS_HEADERS_LIST', 'PREFF_QUALS_HEADERS_LIST', 'LEGAL_NOTIFS_HEADERS_LIST', 'JOB_TITLE_HEADERS_LIST', 'OFFICE_LOC_HEADERS_LIST', 'JOB_DURATION_HEADERS_LIST', 'SUPP_PAY_HEADERS_LIST', 'EDUC_REQS_HEADERS_LIST', 'INTERV_PROC_HEADERS_LIST', 'CORP_SCOPE_HEADERS_LIST', 'POST_DATE_HEADERS_LIST', 'OTHER_HEADERS_LIST'

In [ ]:

print(tag_str)
ha.store_unique_list('REQ_QUALS_HEADERS_LIST', tag_str)

In [5]:

import random

SQL_STR = '''
SELECT navigable_parent
FROM NavigableParents
WHERE
    navigable_parent_id IN (
        SELECT navigable_parent_id
        FROM NavigableParentSequence
        WHERE file_name_id IN (
            SELECT file_name_id
            FROM NavigableParentSequence
            WHERE navigable_parent_id = (
                SELECT navigable_parent_id
                FROM NavigableParents
                WHERE navigable_parent = '<li>401(k)</li>'
                )
            )
        ) AND
    is_header IS NULL'''

In [1576]:

ZMQInteractiveShell_obj = get_ipython()
NAVIGABLE_PARENT_IS_HEADER_DICT = s.load_object('NAVIGABLE_PARENT_IS_HEADER_DICT')
def get_dictionary_code():
    output_str = ''
    cursor_obj = db.execute(SQL_STR)
    row_objs_list = cursor_obj.fetchall()
    child_strs_list = [row_obj['navigable_parent'] for row_obj in row_objs_list]
    tag_str = random.choice(child_strs_list)
    output_str += f'\n# {len(child_strs_list)} to go\n'
    if "'" in tag_str:
        tag_str = tag_str.replace('"', '\\"')
        output_str += f'tag_str = "{tag_str}"\n'
    else:
        output_str += f"tag_str = '{tag_str}'\n"
    output_str += 'NAVIGABLE_PARENT_IS_HEADER_DICT[tag_str] = False\n'
    output_str += 'print(len(NAVIGABLE_PARENT_IS_HEADER_DICT.keys()))\n'
    output_str += 'cursor_obj = db.execute(su.set_is_header0_sql_str, (tag_str,))\n'
    output_str += 'db.commit()\n'
    output_str += 's.store_objects(NAVIGABLE_PARENT_IS_HEADER_DICT=NAVIGABLE_PARENT_IS_HEADER_DICT)'
    
    return output_str

In [ ]:

ZMQInteractiveShell_obj.set_next_input(text=get_dictionary_code(), replace=True)